# Corrigé Projet Maison n° 3 - Session 5

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation ci-dessus).

In [2]:
def df_names_us():
    dfs = [(pd
            .read_csv(f"names/yob{year}.txt", names=["name", "gender", "births"])
            .assign(year=year))
           for year in range(1880, 2022)]
    
    df = (pd.concat(dfs, ignore_index=True)
          .loc[:, ["year", "name", "gender", "births"]]
         )
    
    return df

In [3]:
%%time
df_us = df_names_us()
df_us.shape

Wall time: 3.36 s


(2052781, 4)

In [4]:
# head
df_us.head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [9]:
# with converters
def df_names_fr():
    # genres
    mapping = {1:"M", 2:"F"}
    # read_csv
    df = pd.read_csv("nat2021_csv.zip",
                      sep=";",
                      header=0,
                      names=["gender", "name", "year", "births"],
                      converters={"name": str.title, "gender": mapping.get})
    # clean
    df = (df
          #.loc[lambda df_: (df_.name.str.len() > 1) & (df_.year != "XXXX") & ~(df_.name.str.startswith("_"))]
          .query("""(name.str.len() > 1) and (year != "XXXX") and not name.str.startswith("_")""", engine='python')
          .assign(year=lambda df_: df_.year.astype(int))
          .loc[:, ["year", "name", "gender", "births"]]
          .sort_values(["year", "gender", "births", "name"], ascending=[True, True, False, True])
          .reset_index(drop=True)
         )
    
    return df

In [10]:
%%time
df_fr = df_names_fr()
df_fr.shape

Wall time: 2.35 s


(648330, 4)

In [11]:
# with NA values
def df_names_fr():
    # genres
    mapping = {1:"M", 2:"F"}
    # read_csv
    df = pd.read_csv("nat2021_csv.zip",
                      sep=";",
                      header=0,
                      names=["gender", "name", "year", "births"],
                      na_values={"year":"XXXX", "name":"_PRENOMS_RARES"},
                      keep_default_na=False)
    # clean
    df = (df
          .dropna()
          .loc[lambda df_: df_.name.str.len() > 1]
          .assign(year=lambda df_: df_.year.astype(int),
                  name=lambda df_: df_.name.str.title(),
                  gender=lambda df_: df_.gender.map(mapping)
                 )
          .loc[:, ["year", "name", "gender", "births"]]
          .sort_values(["year", "gender", "births", "name"], ascending=[True, True, False, True])
          .reset_index(drop=True)
         )
    
    return df

In [12]:
%%time
df_fr = df_names_fr()
df_fr.shape

Wall time: 1.83 s


(648330, 4)

In [13]:
# head
df_fr.head()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696


In [14]:
# prénom NA
df_fr.loc[df_fr['name']=='Na']

,year,name,gender,births
404624,2003,Na,F,3


In [15]:
# read_csv default
df = pd.read_csv("nat2021_csv.zip",
                  sep=';',
                  header=0,
                  names=["gender", "name", "year", "births"]
                )

df["name"].isnull().sum()

2

In [16]:
# read_csv mode string sans NA par défaut
df = pd.read_csv("nat2021_csv.zip",
                  sep=';',
                  header=0,
                  names=["gender", "name", "year", "births"],
                  dtype=str,
                  na_values="",
                  keep_default_na=False
                )

df["name"].isnull().sum()

0

In [17]:
# use functools.partial
from functools import partial

load_raw_csv = partial(pd.read_csv, dtype=str, na_values="", keep_default_na=False)

In [18]:
# usage of load_raw_csv
(load_raw_csv("nat2021_csv.zip", sep=";")
 .isna()
 .sum()
)

sexe        0
preusuel    0
annais      0
nombre      0
dtype: int64

In [6]:
# capitalize
"JEAN-MARIE".capitalize()

'Jean-marie'

In [5]:
# title
"JEAN-MARIE".title()

'Jean-Marie'

In [8]:
str.title("JEAN-MARIE")

'Jean-Marie'

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format '%d/%m/%Y') et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [19]:
def df_taux_change(devises):
    df = pd.read_csv("Webstat_Export.csv",
                     sep=";",
                     na_values='-',
                     decimal=',',
                     skiprows=range(1, 6),  # le skiprows permet à l'option "decimal" de fonctionner
                     converters={0: lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='ignore')})

    # extraction des codes iso3 des monnaies
    iso3 = pd.Series(df.columns.tolist()).str.extract('\(([A-Z]{3})\)$', expand=False)
    print(iso3)
    iso3.iloc[0] = 'Date'
    
    df = (df.rename(columns=dict(zip(df.columns, iso3)))
          .loc[:, ['Date'] + devises]
          .dropna()
          .set_index('Date')
          .sort_index()
         )
    
    return df

In [20]:
# exemple
df_tx = df_taux_change(["CHF", "GBP", "USD"])
df_tx.shape

0     NaN
1     AUD
2     BGN
3     BRL
4     CAD
5     CHF
6     CNY
7     CYP
8     CZK
9     DKK
10    EEK
11    GBP
12    HKD
13    HRK
14    HUF
15    IDR
16    ILS
17    NaN
18    ISK
19    JPY
20    KRW
21    LTL
22    LVL
23    MTL
24    MXN
25    MYR
26    NOK
27    NZD
28    PHP
29    PLN
30    RON
31    RUB
32    SEK
33    SGD
34    SIT
35    SKK
36    THB
37    TRY
38    USD
39    ZAR
dtype: object


(6075, 3)

In [21]:
# head
df_tx.head()

,CHF,GBP,USD
Date,,,
1999-01-04,1.6168,0.71110,1.1789
1999-01-05,1.6123,0.71220,1.1790
1999-01-06,1.6116,0.70760,1.1743
1999-01-07,1.6165,0.70585,1.1632
1999-01-08,1.6138,0.70940,1.1659


In [22]:
# index
df_tx.index

DatetimeIndex(['1999-01-04', '1999-01-05', '1999-01-06', '1999-01-07',
               '1999-01-08', '1999-01-11', '1999-01-12', '1999-01-13',
               '1999-01-14', '1999-01-15',
               ...
               '2022-09-07', '2022-09-08', '2022-09-09', '2022-09-12',
               '2022-09-13', '2022-09-14', '2022-09-15', '2022-09-16',
               '2022-09-19', '2022-09-20'],
              dtype='datetime64[ns]', name='Date', length=6075, freq=None)

### Tests

In [23]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [24]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)
    
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... 

0     NaN
1     AUD
2     BGN
3     BRL
4     CAD
5     CHF
6     CNY
7     CYP
8     CZK
9     DKK
10    EEK
11    GBP
12    HKD
13    HRK
14    HUF
15    IDR
16    ILS
17    NaN
18    ISK
19    JPY
20    KRW
21    LTL
22    LVL
23    MTL
24    MXN
25    MYR
26    NOK
27    NZD
28    PHP
29    PLN
30    RON
31    RUB
32    SEK
33    SGD
34    SIT
35    SKK
36    THB
37    TRY
38    USD
39    ZAR
dtype: object


ok

----------------------------------------------------------------------
Ran 3 tests in 6.974s

OK


### Session 5 - Un peu de Data Science...

**Question n° 1**

Pourquoi le `value_counts()` du "gender" donne-t-il un tel écart entre "F" et "M" ?

In [26]:
df_us

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2052776,2021,Zyeire,M,5
2052777,2021,Zyel,M,5
2052778,2021,Zyian,M,5
2052779,2021,Zylar,M,5


In [25]:
# value_counts du gender US

df_us["gender"].value_counts()

F    1209866
M     842915
Name: gender, dtype: int64

In [27]:
# value_counts du gender FR

df_fr["gender"].value_counts()

F    352355
M    295975
Name: gender, dtype: int64

In [28]:
# pct_change

df_us["gender"].value_counts().pct_change()

F         NaN
M   -0.303299
Name: gender, dtype: float64

In [29]:
# pct_change

df_fr["gender"].value_counts().pct_change()

F         NaN
M   -0.160009
Name: gender, dtype: float64

**Question n° 2**

Pourquoi le `value_counts()̀  du "name" donne-t-il ce résultat ?

In [30]:
# value_counts du gender US

df_us["name"].value_counts().head(16)

Leslie     284
Jean       284
Jesse      284
Ollie      284
Johnnie    284
Francis    284
Joseph     284
Sidney     284
John       284
Tommie     284
William    284
Jessie     284
Lee        284
James      284
Marion     284
Charlie    283
Name: name, dtype: int64

In [31]:
# value_counts du gender FR

df_fr["name"].value_counts().head(16)

Alix         244
Ange         244
Camille      244
Dominique    241
Claude       236
Maxime       232
Hyacinthe    231
Marie        213
Gaby         212
Cyrille      209
Elie         204
Andréa       203
France       199
Léandre      198
Stéphane     197
Irène        197
Name: name, dtype: int64

**Question n° 3**

Pourquoi le `value_counts()` du "year" donne-t-il ce résultat ?

In [32]:
# value_counts du gender US

df_us["year"].value_counts().head(20)

2008    35088
2007    34965
2009    34722
2006    34097
2010    34089
2011    33923
2012    33763
2013    33313
2014    33280
2015    33156
2016    33043
2017    32648
2005    32556
2018    32196
2019    32073
2004    32055
2021    31537
2020    31453
2003    31191
2002    30568
Name: year, dtype: int64

In [33]:
# value_counts du gender FR

df_fr["year"].value_counts().head(20)

2014    14332
2012    14233
2013    14168
2015    14127
2017    14087
2016    14051
2018    14015
2019    13874
2011    13807
2020    13612
2021    13499
2010    13395
2009    12456
2008    12297
2007    12147
2006    11954
2005    11403
2004    11006
2003    10446
2002    10077
Name: year, dtype: int64

**Exercice n° 1**

Donnez le prénom qui a été le plus donné lors d'une année.

In [36]:
df_us.nlargest(10, "births")  # df_us.sort(ascending=False).head(10)

,year,name,gender,births
431064,1947,Linda,F,99693
441435,1948,Linda,F,96212
437168,1947,James,M,94762
544671,1957,Michael,M,92723
437169,1947,Robert,M,91651
451676,1949,Linda,F,91019
533204,1956,Michael,M,90715
556249,1958,Michael,M,90574
447476,1948,James,M,88595
510826,1954,Michael,M,88570


In [37]:
df_fr.nlargest(10, "births")  # df_us.sort(ascending=False).head(10)

,year,name,gender,births
95567,1946,Jean,M,53547
1706,1901,Marie,F,52150
3435,1902,Marie,F,51857
97992,1947,Jean,M,51363
5175,1903,Marie,F,50424
6932,1904,Marie,F,50131
8710,1905,Marie,F,48981
0,1900,Marie,F,48713
10545,1906,Marie,F,48447
14231,1908,Marie,F,47460


**Exercice n° 2**

Donnez la liste des prénoms qui contiennent dans l'ordre a, e, i, o et u (US) ou bien a, e, i et o (FR).

In [38]:
df_us.loc[df_us["name"].str.contains(r"a.*e.*i.*o.*u", regex=True), "name"].unique()

array(['Laprecious', 'Markevious', 'Laderious', 'Jakevious',
       'Quanterious', 'Quanterrious', 'Latrevious', 'Jaterrious',
       'Jamerious', 'Jaderious', 'Marquevious', 'Laterrious', 'Jaterious',
       'Dametrious'], dtype=object)

In [46]:
import re

re.search(r"(.*)(a.*)(e.*)(i.*)(o.*)", "Marie-Philomene").groups()

('M', 'ari', 'e-Ph', 'il', 'omene')

In [45]:
re.search(r"(.*)(a.*)(e.*)(i.*)(o.*)", "Marie-Marie-Philomene").groups()

('Marie-M', 'ari', 'e-Ph', 'il', 'omene')

In [ ]:
# ('M', 'arie-Mari', 'e-Ph', 'il', 'omene')

In [39]:
df_fr.loc[df_fr["name"].str.contains(r"a.*e.*i.*o", regex=True), "name"].unique()

array(['Marie-Philomene', 'Marie-Victorine', 'Marie-Victoire',
       'Marie-Simone', 'Marcelino', 'Valerio', 'Valentino', 'Valeriano',
       'Saverio', 'Marie-Nicole', 'Marie-Violaine', 'Marie-Mimose',
       'Marie-Victoria', 'Marcellino', 'Maria-Conception',
       'Marie-Christophe', 'Laurentino', 'Charles-Victor',
       'Charles-Nicolas', 'Marie-Violette', 'Maelio'], dtype=object)

### Méthodes de reshaping (1)

#### pivot_table

La méthode `pivot_table()` prend en argument :
- index : colonne(s) dont les valeurs vont servir d'index à la table pivot
- columns : colonne(s) dont les valeurs vont servir de colonnes à la table pivot
- values : valeurs qui doivent être agrégées selon les modalités de la colonne passée en "index" et de la colonne passée en "columns"
- aggfunc : fonction d'aggrégation des values, par défaut 'mean', 'median', 'min', 'max', 'count', 'sum', 'nunique', et n'importe quelle lambda ou liste de fonctions
- fill_value : valeur en cas d'absence des modalités croisées 

On obtient `NaN` s'il n'y a pas d'occurence croisée.

Excel = tableau croisé dynamique.

In [47]:
# exemple
df = pd.DataFrame([{'A': 1,'B': 1, 'C': 1},
                   {'A': 1,'B': 1, 'C': 2},
                   {'A': 1,'B': 2, 'C': -1},
                   {'A': 2,'B': 1, 'C': 4},
                   {'A': 2,'B': 1, 'C': 5},
                  ])

df

,A,B,C
0,1,1,1
1,1,1,2
2,1,2,-1
3,2,1,4
4,2,1,5


In [48]:
# exemple
tab = df.pivot_table(values='C',
                     index='A',
                     columns='B')
tab

B,1,2
A,,
1,1.5,-1.0
2,4.5,NaN


In [49]:
type(tab)

pandas.core.frame.DataFrame

In [50]:
tab.index

Int64Index([1, 2], dtype='int64', name='A')

In [51]:
tab.columns

Int64Index([1, 2], dtype='int64', name='B')

In [52]:
# exemple
df.pivot_table(values='C',
              index='A',
              columns='B',
              aggfunc=tuple)

B,1,2
A,,
1,"(1, 2)","(-1,)"
2,"(4, 5)",NaN


In [53]:
# exemple
df.pivot_table(values='C',
              index='A',
              columns='B',
              aggfunc='sum')

B,1,2
A,,
1,3.0,-1.0
2,9.0,NaN


**Exercice n° 3**

Calculez une table pivot avec le nombre total de naissances par année et par genre.

In [78]:
df_us.pivot_table(index="year",
                  columns="gender",
                  values="births",
                  aggfunc="sum")

gender,F,M
year,,
1880,90994,110490
1881,91953,100737
1882,107847,113686
1883,112319,104625
1884,129019,114442
...,...,...
2017,1723043,1847191
2018,1696917,1811738
2019,1673030,1788414


In [77]:
df_us.pivot_table(index="year",
                  #columns="gender",
                  values="births",
                  aggfunc="sum")

,births
year,
1880,201484
1881,192690
1882,221533
1883,216944
1884,243461
...,...
2017,3570234
2018,3508655
2019,3461444


In [55]:
df_fr.pivot_table(index="year",
                  columns="gender",
                  values="births",
                  aggfunc="sum")

gender,F,M
year,,
1900,235926,176138
1901,255718,194622
1902,259689,203024
1903,259687,206074
1904,263010,212710
...,...,...
2017,340989,361380
2018,335174,356092
2019,332907,351689


**Exercice n° 4**

Calculez une table pivot avec la diversité des prénoms (nombre de prénoms distincts) par année et par genre.

In [56]:
df_us.pivot_table(index="year",
                  columns="gender",
                  values="name",
                  aggfunc="count")

gender,F,M
year,,
1880,942,1058
1881,938,996
1882,1028,1099
1883,1054,1030
1884,1172,1125
...,...,...
2017,18409,14239
2018,18115,14081
2019,17972,14101


In [58]:
df_fr.pivot_table(index="year",
                  columns="gender",
                  values="name",
                  aggfunc="count").tail(20)

gender,F,M
year,,
2002,5378,4699
2003,5591,4855
2004,5916,5090
2005,6086,5317
2006,6405,5549
2007,6524,5623
2008,6547,5750
2009,6672,5784
2010,7213,6182


#### crosstab

`crosstab()` est une fonction de reshaping qui prend 2 Series partageant le même index (ou 2 colonnes d'un DataFrame) en argument et produit le décompte croisé des occurrences.

On obtient 0 s'il n'y a pas d'occurence croisée.

In [59]:
# exemple
pd.crosstab(df['A'], df['B'])

B,1,2
A,,
1,2,1
2,2,0


In [60]:
# initial
df_us['initial'] = df_us['name'].str[0].str.upper()
# terminal
df_us['terminal'] = df_us['name'].str[-1].str.upper()

df_us.head()

,year,name,gender,births,initial,terminal
0,1880,Mary,F,7065,M,Y
1,1880,Anna,F,2604,A,A
2,1880,Emma,F,2003,E,A
3,1880,Elizabeth,F,1939,E,H
4,1880,Minnie,F,1746,M,E


In [74]:
# crosstab
pd.crosstab(df_us['initial'], df_us['terminal'], normalize="columns", margins=True).sum(axis=0)

terminal
A      1.0
B      1.0
C      1.0
D      1.0
E      1.0
F      1.0
G      1.0
H      1.0
I      1.0
J      1.0
K      1.0
L      1.0
M      1.0
N      1.0
O      1.0
P      1.0
Q      1.0
R      1.0
S      1.0
T      1.0
U      1.0
V      1.0
W      1.0
X      1.0
Y      1.0
Z      1.0
All    1.0
dtype: float64

In [71]:
pd.crosstab?

In [62]:
# pivot_table
df_us.pivot_table(values='name',
                  index='initial',
                  columns='terminal',
                  aggfunc='count',
                  fill_value=0)

terminal,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
initial,,,,,,,,,,,,,,,,,,,,,
A,66332,398,744,3247,31096,526,221,11989,8853,321,...,70,6752,10158,2498,691,346,433,616,9094,810
B,13044,239,157,3022,16117,45,187,2537,2400,79,...,0,2643,2047,3878,354,54,242,310,9276,329
C,29486,430,378,2138,31701,206,803,1966,3858,65,...,0,5773,7076,2252,173,7,37,76,12015,761
D,32302,43,1092,3117,26567,43,371,3898,4753,105,...,28,2250,8309,1416,194,158,421,193,8759,390
E,23524,151,514,2503,18068,52,268,4459,2177,55,...,30,4737,2949,3487,468,15,54,176,4656,142
F,8495,0,291,1526,5676,0,192,976,485,6,...,37,1265,2139,804,63,0,1,338,2192,695
G,10762,63,20,2563,12163,199,509,1041,1911,25,...,0,2281,2453,1956,20,200,0,3,4630,28
H,6138,227,21,2686,6625,100,494,2580,2275,8,...,14,2499,1816,2331,202,8,116,94,5996,129
I,10835,4,760,428,3671,15,228,1781,1111,10,...,56,902,1688,209,212,2,0,0,1763,304


In [75]:
# Z x Z
df_us.loc[(df_us['initial'] == 'Z') & (df_us['terminal'] == 'Z')]

,year,name,gender,births,initial,terminal
1333005,1999,Zyquez,M,5,Z,Z
1551535,2006,Zyquez,M,6,Z,Z
1583884,2007,Zyquez,M,8,Z,Z
1616661,2008,Zyquez,M,11,Z,Z
1721560,2011,Zyquez,M,8,Z,Z
1826667,2014,Zoraiz,M,5,Z,Z
1854509,2015,Zoraiz,M,9,Z,Z
1889406,2016,Zoraiz,M,7,Z,Z
1922037,2017,Zoraiz,M,7,Z,Z
1952438,2018,Zoraiz,M,9,Z,Z


In [76]:
# Z x Z
df_us.loc[df_us['name'].str.startswith('Z') & df_us['name'].str.endswith('z')]

,year,name,gender,births,initial,terminal
1333005,1999,Zyquez,M,5,Z,Z
1551535,2006,Zyquez,M,6,Z,Z
1583884,2007,Zyquez,M,8,Z,Z
1616661,2008,Zyquez,M,11,Z,Z
1721560,2011,Zyquez,M,8,Z,Z
1826667,2014,Zoraiz,M,5,Z,Z
1854509,2015,Zoraiz,M,9,Z,Z
1889406,2016,Zoraiz,M,7,Z,Z
1922037,2017,Zoraiz,M,7,Z,Z
1952438,2018,Zoraiz,M,9,Z,Z
